In [11]:
import requests as re
from gensim.models import KeyedVectors
import random
import numpy as np

model=KeyedVectors.load_word2vec_format("modeleCemantix.bin", binary=True, unicode_errors="ignore")

def send_word(mot):
    r=re.post('https://cemantix.certitudes.org/score',{'word':mot}).json()
    if('error' in r.keys()):
        return -1000
    else:
        return r["score"]
    
def best_start(model,dico_mot):
    url='https://cemantix.certitudes.org/score'
    
    for i in range(50):
        rd=random.randint(1,100000)
        if(model.index_to_key[rd] not in dico_mot.keys()):
            dico_mot[model.index_to_key[rd]]=send_word(model.index_to_key[rd])
    return dico_mot


def find_word(model):
    WORD_to_FIND=""
    word_to_test=[]
    dico_mot={}
    
    dico_mot=best_start(model,dico_mot)
    mot=max(dico_mot, key=dico_mot.get)
    word_to_test=word_to_test+[i[0] for i in model.most_similar(mot,topn=50)][::-1]
    
    while(WORD_to_FIND==""):
    
        if(word_to_test==[]):
            dico_mot=best_start(model,dico_mot)
            mot=max(dico_mot, key=dico_mot.get)
            word_to_test=word_to_test+[i[0] for i in model.most_similar(mot,topn=50)][::-1]
            

        mot_test=word_to_test[-1]
        word_to_test.pop(-1)
        
        if(mot_test not in dico_mot.keys()):
            
            sc=send_word(mot_test)
            if(sc>0.99):
                WORD_to_FIND=mot_test
                
            dico_mot[mot_test]=sc
            if(dico_mot[mot_test]>dico_mot[mot]):
                mot=mot_test
                word_to_test=word_to_test+[i[0] for i in model.most_similar(mot,topn=50)][::-1]

    print("Le mot du jour à trouver est : ",WORD_to_FIND)
    return

find_word(model)

Le mot du jour à trouver est :  concurrent
